In [2]:
from urllib.parse import urlencode, urlparse
from urllib.request import urlopen, urlretrieve
from datetime import datetime
from pandas import DataFrame, read_csv  # Чтобы упаковать результат в стандартный DataFrame.
import os
from datetime import timedelta, date
import socket
import httplib2
import requests
import time


import numpy as np

data_folder = u'E:\\trade_data\\finam\\Ri'

finam_symbols_file_name = data_folder+u"\\finam_symbols.npz"
tickets_file_name=data_folder+u"\\tickets_loadRI.txt"
output_folder=u'E:\\trade_data\\ConstructConvert'

def get_finam_symbols(finam_symbols_file_name, force_reload = False):
    if force_reload:
        finam_symbols = urlopen('http://www.finam.ru/cache/icharts/icharts.js').readlines()
        np.savez_compressed(finam_symbols_file_name,f=finam_symbols)
        return get_symbol_dict(finam_symbols)
    else:
        
        with np.load(finam_symbols_file_name) as f:
              finam_symbols= f["f"]
        return get_symbol_dict(finam_symbols)
    
    return 
def get_symbol_dict(finam_symbols): #получаем имена символов.
    str_id = str(finam_symbols[0])
    str_code = str(finam_symbols[2])
    start = str(str_code).find("[\'") + 2
    end = str_code.find("\']")
    names = str_code[start : end].split('\',\'')
    ids = str_id[str_id.find('[') + 1 : str_id.find(']')].split(',')
    code_names=dict(zip(names, ids))
    return code_names
    

def get_simbol_code(finam_symbols, symbol="SiH5"):
    str_id = str(finam_symbols[0])
    str_code = str(finam_symbols[2])
    start = str(str_code).find("[\'") + 2
    end = str_code.find("\']")
    names = str_code[start : end].split('\',\'')
    ids = str_id[str_id.find('[') + 1 : str_id.find(']')].split(',')
    if symbol in names: # Если искомый тикер symbol имеется в списке names.
        k = 0
        for i, name in enumerate(names):
            if name == symbol:
                 k = i
                 break
        symbol_code = ids[k]
        return symbol_code
    else:
        raise Exception("%s not found\r\n" % symbol)

In [3]:
def create_url(ticket,date):
    FINAM_URL = "http://export.finam.ru/table.csv"
    market = 5
    start_date_str = date
    end_date_str = date
    start_date = datetime.strptime(start_date_str, "%d.%m.%Y").date()
    end_date = datetime.strptime(end_date_str, "%d.%m.%Y").date()
    
    period = 1
    symbol=ticket
    symbol_code = finam_symbols[ticket]
    params =urlencode([('market', market), ('em', symbol_code), ('code', symbol),
                   ('df', start_date.day), ('mf', start_date.month - 1), ('yf', start_date.year),
                   ('from', start_date_str),
                   ('dt', end_date.day), ('mt', end_date.month - 1), ('yt', end_date.year),
                   ('to', end_date_str),
                   ('p', period), ('f', "table"), ('e', ".csv"), ('cn', symbol),
                   ('dtf', 1), ('tmf', 3), ('MSOR', 1), ('mstime', "on"), ('mstimever', 1),
                   ('sep', 3), ('sep2', 1), ('datf', 11), ('at', 1)])
    url = FINAM_URL + params # Полная строка адреса со всеми параметрами.
    return url

def download_ticks_for_date(ticket,date):
    url=create_url(ticket,date)   
    data = read_csv(url,sep=';')
    return data



def make_name(data_folder,ticks_name, tick_date):
    return data_folder+u'\\'+ticks_name+u'\\'+tick_date+u'\\trades.csv'

def save_ticks(data_folder,ticks_name, tick_date, data):
    filename=make_name(data_folder,ticks_name, tick_date)
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    d=data[[ '<TIME>','<ID>', '<LAST>', '<VOL>']]
    d.to_csv(filename,sep=";",index=False,header=False)
    return

def daterange(start_date, end_date):
    for n in range(int (1+(end_date - start_date).days)):
        yield start_date + timedelta(n)

def process_ticks(data_folder,tick_name,start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, "%d.%m.%Y").date()
    end_date = datetime.strptime(end_date_str, "%d.%m.%Y").date()
    for single_date in daterange(start_date, end_date):
        single_date_str=single_date.strftime("%d.%m.%Y")
        #print ("загружаем данные для ",single_date_str)
        
        try:
            file=(make_name(data_folder,tick_name, single_date_str))
            if not os.path.isfile(file ):
                data=download_ticks_for_date(ticket=tick_name,date=single_date_str)
                save_ticks(data_folder=data_folder,ticks_name=tick_name,tick_date=single_date_str,data=data)
                print(file, "ok!")
        except: 
            print ("данные не загружены! ",single_date_str)    
    

In [4]:
print ("загрузка имен тикетов...")
finam_symbols=get_finam_symbols(finam_symbols_file_name, force_reload = False)

print ("Загрузка имен и дат тикетов...")
tickets =read_csv(tickets_file_name,skiprows=0,delimiter="\t")


загрузка имен тикетов...
Загрузка имен и дат тикетов...


In [5]:
tickets

,Name,Start,End
0,RIH5,15.12.2014,16.03.2015
1,RIM5,17.03.2015,15.06.2015
2,RIU5,16.06.2015,15.09.2015
3,RIZ5,16.09.2015,15.12.2015
4,RIH6,16.12.2015,15.03.2016
5,RIM6,16.03.2016,15.06.2016
6,RIU6,16.06.2016,15.09.2016
7,RIZ6,16.09.2016,15.12.2016
8,RIH7,16.12.2016,16.03.2017
9,RIM7,17.03.2017,15.06.2017


In [7]:
for index, row in tickets.iterrows():
    print (row["Name"], row["Start"], row['End'],finam_symbols[row["Name"]]  )
    process_ticks(data_folder=data_folder, tick_name=row["Name"], start_date_str=row["Start"], end_date_str=row["End"] )



RIH5 15.12.2014 16.03.2015 152736
данные не загружены!  20.12.2014
данные не загружены!  21.12.2014
данные не загружены!  27.12.2014
данные не загружены!  28.12.2014
данные не загружены!  31.12.2014
данные не загружены!  01.01.2015
данные не загружены!  02.01.2015
данные не загружены!  03.01.2015
данные не загружены!  04.01.2015
данные не загружены!  07.01.2015
данные не загружены!  10.01.2015
данные не загружены!  11.01.2015
данные не загружены!  17.01.2015
данные не загружены!  18.01.2015
данные не загружены!  24.01.2015
данные не загружены!  25.01.2015
данные не загружены!  31.01.2015
данные не загружены!  01.02.2015
данные не загружены!  07.02.2015
данные не загружены!  08.02.2015
данные не загружены!  14.02.2015
данные не загружены!  15.02.2015
данные не загружены!  21.02.2015
данные не загружены!  22.02.2015
данные не загружены!  23.02.2015
данные не загружены!  28.02.2015
данные не загружены!  01.03.2015
данные не загружены!  07.03.2015
данные не загружены!  08.03.2015
данные не

## перенос файлов для расчетов векторов

In [8]:
import shutil
def load_folders(root):
    if os.path.exists(root):
        print ('ok')
        d=[os.path.join(root, d) for d in sorted(os.listdir(root))  if os.path.isdir(os.path.join(root, d))]
        return d
    else :
        print (' not ok')
        
folders = load_folders(data_folder)

def date_by_folder(folder_path):
    foldername =os.path.basename(folder_path)
    return datetime.strptime(foldername, "%d.%m.%Y").date().strftime("%Y.%m.%d")
    


ok


In [9]:
folders

['E:\\trade_data\\finam\\Ri\\RIH5',
 'E:\\trade_data\\finam\\Ri\\RIH6',
 'E:\\trade_data\\finam\\Ri\\RIH7',
 'E:\\trade_data\\finam\\Ri\\RIM5',
 'E:\\trade_data\\finam\\Ri\\RIM6',
 'E:\\trade_data\\finam\\Ri\\RIU5',
 'E:\\trade_data\\finam\\Ri\\RIU6',
 'E:\\trade_data\\finam\\Ri\\RIU8',
 'E:\\trade_data\\finam\\Ri\\RIZ5',
 'E:\\trade_data\\finam\\Ri\\RIZ6']

In [10]:
#удаление пустых папок и папок с нулевыми файлами
for folder in folders:
    dates_folders = load_folders(folder)
    for date_folder in dates_folders:
        file_name=date_folder+u'\\trades.csv'
        print (file_name)
        if os.path.isfile(file_name):
            if os.stat(file_name).st_size ==0:
                print ("zero file")
                shutil.rmtree(date_folder)
        else:
            print ("folder empty")
            shutil.rmtree(date_folder)

ok
E:\trade_data\finam\Ri\RIH5\02.02.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\02.03.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\03.02.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\03.03.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\04.02.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\04.03.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\05.01.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\05.02.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\05.03.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\06.01.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\06.02.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\06.03.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\08.01.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\09.01.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\09.02.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\10.02.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\10.03.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\11.02.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\11.03.2015\trades.csv
E:\trade_data\finam\Ri\RIH5\12.01.2015\trades.c

In [11]:
#перенос файлов с пампками
for folder in folders:
    dates_folders = load_folders(folder)
    for date_folder in dates_folders:
        file_name=date_folder+u'\\trades.csv'
        print (file_name)
        if os.path.isfile(file_name):
            if os.stat(file_name).st_size ==0:
                print ("zero file")
            else:
                new_name=output_folder+u'\\'+date_by_folder(date_folder)+u'\\trades.csv'
                print (new_name)
                os.mkdir(os.path.dirname(new_name))
                shutil.copy(date_folder+u'\\trades.csv',  new_name)
        else:
            print ("folder empty")

ok
E:\trade_data\finam\Ri\RIH5\02.02.2015\trades.csv
E:\trade_data\ConstructConvert\2015.02.02\trades.csv
E:\trade_data\finam\Ri\RIH5\02.03.2015\trades.csv
E:\trade_data\ConstructConvert\2015.03.02\trades.csv
E:\trade_data\finam\Ri\RIH5\03.02.2015\trades.csv
E:\trade_data\ConstructConvert\2015.02.03\trades.csv
E:\trade_data\finam\Ri\RIH5\03.03.2015\trades.csv
E:\trade_data\ConstructConvert\2015.03.03\trades.csv
E:\trade_data\finam\Ri\RIH5\04.02.2015\trades.csv
E:\trade_data\ConstructConvert\2015.02.04\trades.csv
E:\trade_data\finam\Ri\RIH5\04.03.2015\trades.csv
E:\trade_data\ConstructConvert\2015.03.04\trades.csv
E:\trade_data\finam\Ri\RIH5\05.01.2015\trades.csv
E:\trade_data\ConstructConvert\2015.01.05\trades.csv
E:\trade_data\finam\Ri\RIH5\05.02.2015\trades.csv
E:\trade_data\ConstructConvert\2015.02.05\trades.csv
E:\trade_data\finam\Ri\RIH5\05.03.2015\trades.csv
E:\trade_data\ConstructConvert\2015.03.05\trades.csv
E:\trade_data\finam\Ri\RIH5\06.01.2015\trades.csv
E:\trade_data\Constr

FileExistsError: [WinError 183] Невозможно создать файл, так как он уже существует: 'E:\\trade_data\\ConstructConvert\\2017.02.01'

In [ ]:
#выгрузка без субдиректориев
for folder in folders:
    dates_folders = load_folders(folder)
    for date_folder in dates_folders:
        file_name=date_folder+u'\\trades.csv'
        print (file_name)
        if os.path.isfile(file_name):
            if os.stat(file_name).st_size ==0:
                print ("zero file")
            else:    
                new_name=output_folder+u'\\'+date_by_folder(date_folder)+u'_trades.csv'
                print (new_name)
              
                shutil.copy(date_folder+u'\\trades.csv',  new_name)
        else:
            print ("folder empty")